In [4]:
from config import config_qa_net
import loader
config = config_qa_net.config
embedding_np = loader.load_w2v(config.embedding_path)

In [5]:
param ={
    'dropout_p':0.1,
    'encoder_dropout_p':0.1,
    'embedding': embedding_np,
    'hidden_size': 128
}

In [6]:
a = Model(param)

AttributeError: 'NoneType' object has no attribute 'fill_'

In [9]:
a.cuda()

Model(
  (embedding): ExtendEmbedding(
    (sd_embedding): Embedding(
      (embedding): Embedding(113315, 256, padding_idx=0)
    )
    (tag_embedding): Embedding(60, 4, padding_idx=0)
  )
  (content_conv): DepthwiseSeparableConv(
    (depthwise_conv): Conv1d(262, 262, kernel_size=(5,), stride=(1,), padding=(2,), groups=262)
    (pointwise_conv): Conv1d(262, 128, kernel_size=(1,), stride=(1,))
  )
  (question_conv): DepthwiseSeparableConv(
    (depthwise_conv): Conv1d(260, 260, kernel_size=(5,), stride=(1,), padding=(2,), groups=260)
    (pointwise_conv): Conv1d(260, 128, kernel_size=(1,), stride=(1,))
  )
  (c_enc): EncoderBlock(
    (convs): ModuleList(
      (0): DepthwiseSeparableConv(
        (depthwise_conv): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,), groups=128)
        (pointwise_conv): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      )
      (1): DepthwiseSeparableConv(
        (depthwise_conv): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(

In [11]:
for name, p in a.named_parameters():
    print(name, p.nelement())

embedding.sd_embedding.embedding.weight 29008640
embedding.tag_embedding.weight 240
content_conv.depthwise_conv.weight 1310
content_conv.depthwise_conv.bias 262
content_conv.pointwise_conv.weight 33536
content_conv.pointwise_conv.bias 128
question_conv.depthwise_conv.weight 1300
question_conv.depthwise_conv.bias 260
question_conv.pointwise_conv.weight 33280
question_conv.pointwise_conv.bias 128
c_enc.convs.0.depthwise_conv.weight 896
c_enc.convs.0.depthwise_conv.bias 128
c_enc.convs.0.pointwise_conv.weight 16384
c_enc.convs.0.pointwise_conv.bias 128
c_enc.convs.1.depthwise_conv.weight 896
c_enc.convs.1.depthwise_conv.bias 128
c_enc.convs.1.pointwise_conv.weight 16384
c_enc.convs.1.pointwise_conv.bias 128
c_enc.convs.2.depthwise_conv.weight 896
c_enc.convs.2.depthwise_conv.bias 128
c_enc.convs.2.pointwise_conv.weight 16384
c_enc.convs.2.pointwise_conv.bias 128
c_enc.convs.3.depthwise_conv.weight 896
c_enc.convs.3.depthwise_conv.bias 128
c_enc.convs.3.pointwise_conv.weight 16384
c_enc.co